Set cuda

In [1]:
import torch
import os
import sys
device=7 # No other value works. known bug in Jupyter.

if torch.cuda.is_available():
    torch.cuda.set_device(device)

imports

In [2]:
%env WANDB_NOTEBOOK_NAME=JUPYTER

import wandb
import math

import os
from myhelpers import TrialStatistics
from HGNN.train import CNN, dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from tqdm import tqdm
from tqdm.auto import trange
wandb.login()

env: WANDB_NOTEBOOK_NAME=JUPYTER


wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)


True

sweep params

In [3]:
# sweep_config = {
#     'method': 'random'
# }

sweep_config = {
    'method': 'bayes',
    'early_terminate': {
       'type': 'hyperband',
       'min_iter': 8   
    }
}
metric = {
    'name': 'validation_fine_f1',
    'goal': 'maximize'   
    }
sweep_config['metric'] = metric

In [4]:
project="HGNN_sweep"
params = {
    "image_path":  {
        'value': "Curated4/Easy_50"
    },
    "suffix":{
        'value': None
    },
    
    # dataset
    "img_res": {
        'value': 448
    }, 
    "augmented": {
        'value': True
    }, 

    # training
    "batchSize": {
          'values': [16, 32, 64, 128]
        }, 
    "learning_rate":{
        'distribution': 'log_uniform',
        'min': math.log(0.00005),
        'max': math.log(0.5),
      },
    "numOfTrials":{
        'value': 1
    },
    "fc_layers": {
          'values': [1, 2, 3]
        },
    "modelType":{
        'value':"HGNN_add"
    }, #BB DISCO DSN HGNN HGNN_add HGNN_cat
    "lambda": {
        'distribution': 'log_uniform',
        'min': math.log(0.005),
        'max': math.log(0.5),
      },
    "unsupervisedOnTest": {
        'value': False
    },
    "tl_model": {
        'value': 'ResNet18'
    }, 
    "link_layer":  {
        'values': ["avgpool","layer3"]
    }, 
    
    "adaptive_smoothing": {
        'values': [True,False]
    },
    "adaptive_lambda": {
        'distribution': 'log_uniform',
        'min': math.log(0.001),
        'max': math.log(0.1),
      }, 
    "adaptive_alpha": {
        'min': 0.5,
        'max': 0.99,
      }, 
    
    "noSpeciesBackprop":{
        'values': [True,False]
    },
}

sweep_config['parameters'] = params

train function

In [5]:

# python3 HGNN/code/HGNN/HGNN/train/train.py --cuda=6 --name="biology_paper_curated4_Medium_backprop_experiment" -- -- --detailed

experimentsPath="/raid/elhamod/Fish/experiments/"
dataPath="/raid/elhamod/Fish/"
experimentName = project

def train(config=None):
    run = wandb.init()

    try:
        experiment_params=wandb.config if config is None else config
        experiment_params = dict(experiment_params)
        config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
        print(experiment_params)
        experimentHash =TrialStatistics.getTrialName(experiment_params)

        # load images 
        experimentPathAndName = os.path.join(experimentsPath, experimentName)
        datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
        datasetManager.updateParams(config_parser.fixPaths(experiment_params))
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        architecture = {
            "fine": len(train_loader.dataset.csv_processor.getFineList()),
            "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
        }

        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            trialHash = TrialStatistics.getTrialName(experiment_params, i)

            row_information = {
                'experimentName': experimentName,
                'modelName': modelName,
                'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
                'experimentHash': experimentHash,
                'trialHash': trialHash
            }
            row_information = {**row_information, **experiment_params} 
            print(row_information)


            # Train/Load model
            model = CNN.create_model(architecture, experiment_params, device=device)

            if os.path.exists(CNN.getModelFile(trialName)):
                print("Model {0} found!".format(trialName))
            else:
                CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=False)

            run.finish()
    except:
        print('terminated! Error:', sys.exc_info()[0])

Call sweeper!

In [10]:
# You can call with your own configs if you want:
# train(config={...})

In [11]:
# Or call the sweeper and let it do the magic!
sweep_id = wandb.sweep(sweep_config, project=project)

Create sweep with ID: 25xa83hy
Sweep URL: https://wandb.ai/mndhamod/HGNN_sweep/sweeps/25xa83hy


In [8]:
wandb.agent(sweep_id, function=train, count=100)
# !CUDA_VISIBLE_DEVICES=1 wandb agent k7cnzayi --count 100 #train.py from program?

wandb: Agent Starting Run: qimmtli8 with config:
wandb: 	adaptive_alpha: 0.6664130302929847
wandb: 	adaptive_lambda: 0.07085604986584583
wandb: 	adaptive_smoothing: False
wandb: 	augmented: True
wandb: 	batchSize: 16
wandb: 	fc_layers: 3
wandb: 	image_path: Curated4/Easy_50
wandb: 	img_res: 448
wandb: 	lambda: 0.007784220820224443
wandb: 	learning_rate: 0.2594347010050439
wandb: 	link_layer: layer3
wandb: 	modelType: HGNN_add
wandb: 	noSpeciesBackprop: False
wandb: 	numOfTrials: 1
wandb: 	suffix: None
wandb: 	tl_model: ResNet18
wandb: 	unsupervisedOnTest: False


{'adaptive_alpha': 0.6664130302929847, 'adaptive_lambda': 0.07085604986584583, 'adaptive_smoothing': False, 'augmented': True, 'batchSize': 16, 'fc_layers': 3, 'image_path': 'Curated4/Easy_50', 'img_res': 448, 'lambda': 0.007784220820224443, 'learning_rate': 0.2594347010050439, 'link_layer': 'layer3', 'modelType': 'HGNN_add', 'noSpeciesBackprop': False, 'numOfTrials': 1, 'suffix': None, 'tl_model': 'ResNet18', 'unsupervisedOnTest': False}
Creating datasets...
Creating datasets... Done.
Creating loaders...
Creating loaders... Done.


{'experimentName': 'HGNN_sweep', 'modelName': 'models/06d04b31d9229553d1b36fc7d8f71ec15969a0da8cc3d59b7ba673b6', 'datasetName': 'datasplits/7c7513bdfb4e5577fef1c3ec0fa6452d30d87dbc95f258a7c86dd76b', 'experimentHash': '319f39801bac5153b1161d4e740d197c7b8284ec52e7f169ff242f4d', 'trialHash': '06d04b31d9229553d1b36fc7d8f71ec15969a0da8cc3d59b7ba673b6', 'adaptive_alpha': 0.6664130302929847, 'adaptive_lambda': 0.07085604986584583, 'adaptive_smoothing': False, 'augmented': True, 'batchSize': 16, 'fc_layers': 3, 'image_path': 'Curated4/Easy_50', 'img_res': 448, 'lambda': 0.007784220820224443, 'learning_rate': 0.2594347010050439, 'link_layer': 'layer3', 'modelType': 'HGNN_add', 'noSpeciesBackprop': False, 'numOfTrials': 1, 'suffix': None, 'tl_model': 'ResNet18', 'unsupervisedOnTest': False}


iteration:   0%|          | 0/500 [00:00<?, ?it/s]

Training started...


...

iteration:   3%|▎         | 14/500 [13:47<7:43:13, 57.19s/it, min_val_loss=1.28, train=1, val=0.774, val_loss=3.28]   

Early stopping
total number of epochs:  13


iteration:   3%|▎         | 14/500 [13:49<7:59:55, 59.25s/it, min_val_loss=1.28, train=1, val=0.774, val_loss=3.28]


loss,0.60864
batch,265
epoch,13
loss_fine,0.53588
loss_coarse,0.92816
lambda_fine,0.81452
lambda_coarse,0.18548
_step,279
_runtime,829
_timestamp,1610026185
validation_fine_f1,0.77433


loss,█▇▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▁▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
batch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss_fine,█▇▆▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_coarse,█▆▄▃▃▂▃▂▃▂▂▃▂▂▂▂▃▂▃▁▂▂▂▂▃▂▂▁▂▂▁▃▁▁▁▃▁▁▂▂
lambda_fine,▇▃▁▂▃▁▄▃▄▃▄▅▄▅▅▆▆▅▇▄▅▅▅▆█▆▆▄▅▇▅▇▅▄▅▇▅▄▆▇
lambda_coarse,▂▆█▇▆█▅▆▅▆▅▄▅▄▄▃▃▄▂▅▄▄▄▃▁▃▃▅▄▂▄▂▄▅▄▂▄▅▃▂
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
validation_fine_f1,▁▅▆▇▇▇▇███████
